In [1]:
!mkdir -p /content/project/data
!mkdir -p /content/project/faiss_index

In [2]:
!ls -R /content/project

/content/project:
data  faiss_index

/content/project/data:

/content/project/faiss_index:


In [3]:
!pip install -q langchain langchain-community faiss-cpu sentence-transformers transformers sacremoses streamlit bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 134.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 128.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/

In [4]:
!pip install -q torchvision

In [5]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 58.3 MB/s eta 0:00:00


In [6]:
import fitz  # PyMuPDF
import os

# Path to the folder with PDFs
pdf_folder = "/content/project/data"

# List to hold all extracted text
all_texts = []

# Loop through each PDF in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)
        print(f"📘 Extracting from: {filename}")

        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()

        all_texts.append(text)

print(f"\n✅ Extracted text from {len(all_texts)} PDF(s).")

📘 Extracting from: World-Alzheimer-Report-2024.pdf
📘 Extracting from: nihms-1674325.pdf
📘 Extracting from: 1980-5764-DN-19-e20240204.pdf
📘 Extracting from: nia-alzheimers-fact-sheet.pdf

✅ Extracted text from 4 PDF(s).


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Create Document objects from the raw text
documents = [Document(page_content=text) for text in all_texts]

# Splitter configuration
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

# Perform the split
chunked_docs = text_splitter.split_documents(documents)

print(f"✅ Total chunks created: {len(chunked_docs)}")
print(f"🧩 Example chunk:\n\n{chunked_docs[0].page_content[:500]}")

✅ Total chunks created: 1650
🧩 Example chunk:

World Alzheimer Report 2024
Global changes in attitudes to dementia
World Alzheimer Report 2024
Global changes in attitudes to dementia
4
 
ALZHEIMER’S DISEASE INTERNATIONAL | WORLD ALZHEIMER REPORT 2024
Authors
Alzheimer’s Disease International
Attitudes to dementia survey:
Dr Sara Evans-Lacko, Elisa Aguzzoli, Dr Sanna Read, Adelina Comas-Herrera, Dr Nicolas Farina
Contributors:


In [8]:
# First chunk
print("🔹 First chunk:\n", chunked_docs[0].page_content)

# Random chunk
import random
i = random.randint(0, len(chunked_docs)-1)
print(f"\n🔹 Random chunk #{i}:\n", chunked_docs[i].page_content)

🔹 First chunk:
 World Alzheimer Report 2024
Global changes in attitudes to dementia
World Alzheimer Report 2024
Global changes in attitudes to dementia
4
 
ALZHEIMER’S DISEASE INTERNATIONAL | WORLD ALZHEIMER REPORT 2024
Authors
Alzheimer’s Disease International
Attitudes to dementia survey:
Dr Sara Evans-Lacko, Elisa Aguzzoli, Dr Sanna Read, Adelina Comas-Herrera, Dr Nicolas Farina
Contributors:

🔹 Random chunk #601:
 Sajatovic, M. (2018). A systematic review of dementia-related stigma research: 
Can we move the stigma dial? The American Journal of Geriatric Psychiatry, 
316–331.
13.	Tolhurst, E., Weicht, B., & Runacres, J. (2023). Sustaining relational subjectivity: 
The experience of women with dementia. Sociology of Health & Illness, 503–521.
14.	Weidner, W., Barbarino, P., & Lynch, C. (2021). Women and dementia policy: 
redressing imbalance through gender transformative policies. In M. Ferretti, A.


In [9]:
!pip install -U langchain-huggingface

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Load biomedical sentence embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="pritamdeka/S-PubMedBERT-MS-MARCO"
)

print("✅ Biomedical embedding model loaded.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Biomedical embedding model loaded.


In [11]:
# Create the FAISS vector index
db = FAISS.from_documents(chunked_docs, embedding_model)

# Save the FAISS index locally
db.save_local("/content/project/faiss_index")

print(f"✅ Vector index created and saved! Total chunks embedded: {len(chunked_docs)}")


✅ Vector index created and saved! Total chunks embedded: 1650


In [12]:
!pip install -q accelerate

In [13]:
!pip install -q dotenv

In [14]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
login(os.getenv("HF_TOKEN"))

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from transformers import BitsAndBytesConfig

# Replace with the model you prefer
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=text_pipeline)
print("✅ Mistral-Instruct model loaded and ready.")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Mistral-Instruct model loaded and ready.


/tmp/ipython-input-15-2746064465.py:26: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_pipeline)


In [16]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Load the FAISS index
db = FAISS.load_local(
    "/content/project/faiss_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a trusted biomedical research assistant specialized in neurodegenerative diseases.

Use only the information provided in the context to answer the question.

Do not use prior knowledge or speculate. If the answer is not in the context, say "I don't know."

Answer concisely and professionally, suitable for clinical or academic use.

Context:
{context}

Question:
{question}

Answer:"""
)

# ✅ Build the RAG chain using the custom prompt
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 8}),
    return_source_documents=False,
    chain_type_kwargs={"prompt": custom_prompt}
)

print("✅ RAG pipeline is live and ready for Q&A!")

✅ RAG pipeline is live and ready for Q&A!


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
def clean_output(response_text):
    if "Answer:" in response_text:
        response_text = response_text.split("Answer:")[-1]
    if "Helpful Answer:" in response_text:
        response_text = response_text.split("Helpful Answer:")[-1]
    if "Unhelpful" in response_text:
        response_text = response_text.split("Unhelpful")[0]
    return response_text.strip()

In [ ]:
while True:
    query = input("Ask a question (or type 'exit'): ")
    if query.lower() == 'exit':
        break
    response = rag_chain.invoke(query)
    print("Answer:", clean_output(response['result']))


Ask a question (or type 'exit'): what is the first common noticable symptom of alzhimers


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer: The first common noticable symptom of Alzheimer's is memory problems.
Ask a question (or type 'exit'): what kind of memory problems?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer: The text indicates that memory problems are one of the first signs of cognitive impairment related to Alzheimer's. Therefore, the type of memory problems associated with Alzheimer's is primarily memory loss.
Ask a question (or type 'exit'): exit


##Evaluation

In [ ]:
from project.alzheimers_test_set import test_set
from difflib import SequenceMatcher

def evaluate_response(generated, expected):
    return SequenceMatcher(None, generated.lower(), expected.lower()).ratio()

results = []

for item in test_set:
    query = item["question"]
    expected = item["expected_answer"]

    response = rag_chain.invoke(query)
    generated = clean_output(response["result"])

    score = evaluate_response(generated, expected)

    results.append({
        "question": query,
        "expected": expected,
        "generated": generated,
        "score": round(score, 2)
    })

# Print individual results
for res in results:
    print(f"\n🧠 Q: {res['question']}")
    print(f"✅ Expected: {res['expected']}")
    print(f"🤖 Generated: {res['generated']}")
    print(f"📈 Similarity Score: {res['score']}\n{'-'*70}")

# Print overall average score
average_score = sum([r["score"] for r in results]) / len(results)
print(f"\n🎯 Final Average Similarity Score: {round(average_score * 100, 2)}%")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting


🧠 Q: What are the early symptoms of Alzheimer's disease?
✅ Expected: Early symptoms often include difficulties with word-finding, spatial awareness issues, and impaired reasoning or judgment. These symptoms may appear before noticeable memory loss.
🤖 Generated: The early symptoms of Alzheimer's disease include memory problems, such as difficulty remembering recent events or familiar names, as well as non-memory aspects of cognition, like word-finding, vision/spatial issues, and impaired reasoning or judgment. Researchers are studying biomarkers to help detect these early changes in the brain. Additionally, some people with memory problems may have a condition called mild cognitive impairment (MCI), where symptoms do not significantly interfere with everyday life. Early diagnosis and treatment could potentially help preserve daily functioning and aid families in planning for the future.
📈 Similarity Score: 0.35
----------------------------------------------------------------------

🧠 Q

In [ ]:
import csv

csv_path = "/content/project/evaluation_results.csv"  # change if needed

with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["question", "expected", "generated", "score"])
    writer.writeheader()
    for row in results:
        writer.writerow(row)

print(f"📁 Evaluation results saved to: {csv_path}")

In [ ]:
!pip install streamlit
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 2s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [ ]:
!curl https://loca.lt/mytunnelpassword


35.204.96.131

In [ ]:
%%writefile /content/launch_streamlit.sh
!streamlit run /content/project/app.py --server.port 8501


Writing /content/launch_streamlit.sh


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!streamlit run /content/project/app.py --server.port 8501 & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.204.96.131:8501

⠧⠇⠏⠋⠙⠹your url is: https://cute-games-push.loca.lt
2025-07-15 15:52:21.413243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752594741.614300   12654 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752594741.671886   12654 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/langchain/llms/__init__.py:549: LangChainDeprecationWarning: Importing LLMs from langchain is deprecated. Importing from langchain will no longer be supported as of

In [ ]:
!pip freeze > requirements.txt

In [ ]:
test_set = [
    {
        "question": "What are the early symptoms of Alzheimer's disease?",
        "expected_answer": "Early symptoms often include difficulties with word-finding, spatial awareness issues, and impaired reasoning or judgment. These symptoms may appear before noticeable memory loss."
    },
    {
        "question": "Can Alzheimer's be reversed?",
        "expected_answer": "Currently, Alzheimer’s cannot be reversed. However, early treatment can help manage symptoms and slow progression in some individuals."
    },
    {
        "question": "How does Alzheimer's affect memory?",
        "expected_answer": "Alzheimer’s progressively damages memory by first affecting short-term recall and gradually impairing long-term memory as the disease advances. This impacts daily activities and independence."
    },
    {
        "question": "What are biomarkers in Alzheimer's diagnosis?",
        "expected_answer": "Biomarkers are biological indicators like amyloid-beta or tau levels found in brain scans, cerebrospinal fluid, or blood. They help confirm Alzheimer’s and differentiate it from other conditions."
    },
    {
        "question": "Which stage is most commonly diagnosed in Alzheimer's?",
        "expected_answer": "Most individuals are diagnosed during the mild to moderate stages when symptoms become more noticeable. These include memory lapses and functional impairments in daily life."
    },
    {
        "question": "What are the behavioral changes in Alzheimer's?",
        "expected_answer": "Behavioral changes in Alzheimer's may include irritability, aggression, anxiety, and mood swings. These symptoms often become more pronounced as the disease progresses."
    },
    {
        "question": "How is Alzheimer’s diagnosed?",
        "expected_answer": "Alzheimer’s is diagnosed using cognitive assessments, clinical history, and imaging tools such as MRI and PET scans. Biomarker testing may also be used for confirmation."
    },
    {
        "question": "What are some treatable conditions that mimic Alzheimer’s?",
        "expected_answer": "Sleep disorders, medication side effects, infections, vitamin deficiencies, and depression can mimic Alzheimer’s. These conditions should be ruled out first as they may be reversible."
    },
    {
        "question": "How does Alzheimer's impact language?",
        "expected_answer": "People with Alzheimer’s may struggle to find the right words, follow conversations, or understand spoken and written language. These issues worsen as the disease progresses."
    },
    {
        "question": "What are the causes of Alzheimer’s?",
        "expected_answer": "Alzheimer’s is caused by a combination of genetic, environmental, and lifestyle factors. The accumulation of amyloid plaques and tau tangles in the brain is a hallmark of the disease."
    },
    {
        "question": "Is Alzheimer's hereditary?",
        "expected_answer": "Yes, Alzheimer’s can run in families, particularly early-onset cases. However, most cases are sporadic, and having a genetic risk doesn’t guarantee developing the disease."
    },
    {
        "question": "How does Alzheimer's differ from dementia?",
        "expected_answer": "Alzheimer’s is the most common form of dementia, but not all dementia is Alzheimer’s. Other types include vascular dementia and Lewy body dementia, which have distinct features."
    },
    {
        "question": "Can Alzheimer’s affect vision?",
        "expected_answer": "Yes, Alzheimer’s can cause visual-spatial difficulties, such as trouble judging distances or recognizing objects. These are not due to eye problems but brain processing issues."
    },
    {
        "question": "What stage of Alzheimer's involves personality changes?",
        "expected_answer": "Personality changes such as increased suspicion or withdrawal often occur in the moderate stage of Alzheimer’s. These changes may result from frustration or confusion."
    },
    {
        "question": "What is the role of caregivers in Alzheimer’s management?",
        "expected_answer": "Caregivers provide essential support by assisting with daily tasks, ensuring medication adherence, and managing behavioral symptoms. They also offer emotional support to patients."
    },
    {
        "question": "How does Alzheimer’s affect reasoning and judgment?",
        "expected_answer": "Alzheimer’s leads to poor decision-making, difficulty with problem-solving, and impaired ability to plan. This can affect financial management and safety awareness."
    },
    {
        "question": "What medications are used to treat Alzheimer’s?",
        "expected_answer": "Common medications include donepezil, rivastigmine, and memantine. These drugs help manage symptoms but do not stop or reverse disease progression."
    },
    {
        "question": "What lifestyle changes can help manage Alzheimer’s?",
        "expected_answer": "Engaging in physical activity, eating a brain-healthy diet, and maintaining cognitive engagement may help slow Alzheimer’s progression. Social interactions also provide benefits."
    },
    {
        "question": "Can early diagnosis help with Alzheimer’s treatment?",
        "expected_answer": "Yes, early diagnosis allows for timely treatment, better care planning, and access to support services. It may also help patients participate in clinical trials."
    },
    {
        "question": "What is mild cognitive impairment (MCI)?",
        "expected_answer": "MCI is a condition where individuals experience noticeable cognitive decline without significant interference in daily life. It can be a precursor to Alzheimer’s but not always."
    }
]


In [ ]:
import streamlit as st
from rag_pipeline import load_rag_chain, query_rag

# --- 🌐 Page Setup ---
st.set_page_config(
    page_title="Alzheimer's QnA App",
    page_icon="🧠",
    layout="centered",
    initial_sidebar_state="collapsed"
)

# --- 🌈 Styling ---
st.markdown("""
    <style>
        .stApp {
            background-color: #f3f4f6;
            padding-top: 2rem;
        }
        .title-text {
            font-size: 40px;
            text-align: center;
            font-weight: bold;
            color: #4a4a4a;
        }
        .subtitle-text {
            font-size: 20px;
            text-align: center;
            color: #6c757d;
            margin-bottom: 30px;
        }
        .answer-box {
            background-color: #ffffff;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
            font-size: 18px;
            color: #333333;
        }
    </style>
""", unsafe_allow_html=True)

# --- 🧠 Header ---
st.markdown('<div class="title-text">🧠 Alzheimer\'s Disease Q&A</div>', unsafe_allow_html=True)
st.markdown('<div class="subtitle-text">Powered by Mistral + LangChain RAG + BioMed Embeddings</div>', unsafe_allow_html=True)

# --- 📦 Load RAG Pipeline ---
with st.spinner("Loading RAG system. Please wait..."):
    rag_chain = load_rag_chain()

st.success("✅ Enter your query below.")

# --- 💬 Input Box ---
query = st.text_input(
    "💬 Ask a medical question",
    placeholder="e.g., What are the early symptoms of Alzheimer?"
)

# --- 🚀 Answer Area ---
if query:
    with st.spinner("🔍 Retrieving answer..."):
        answer = query_rag(rag_chain, query)

    st.markdown("### ✅ Answer:")
    st.markdown(f'<div class="answer-box">{answer}</div>', unsafe_allow_html=True)


In [ ]:

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
import torch

# Load LLM
def load_llm():
    model_id = "mistralai/Mistral-7B-Instruct-v0.2"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
    return HuggingFacePipeline(pipeline=pipe)

# Load vectorstore
def load_vectorstore():
    embedding_model = HuggingFaceEmbeddings(model_name="pritamdeka/S-PubMedBERT-MS-MARCO")
    return FAISS.load_local("project/faiss_index", embedding_model, allow_dangerous_deserialization=True)

# Custom Prompt
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are a helpful biomedical assistant. Answer based only on the given context. If the answer isn't there, say "I don't know".

{context}
Question: {question}
Answer:"""
)

# Load RAG pipeline
def load_rag_chain():
    llm = load_llm()
    db = load_vectorstore()
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=db.as_retriever(search_kwargs={"k": 4}),
        chain_type_kwargs={"prompt": custom_prompt},
        return_source_documents=False
    )

# Query Function
def query_rag(rag_chain, question):
    response = rag_chain.invoke({"query": question})
    return response["result"].strip()
